In [91]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [92]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

# Q1. Downloading the data

In [93]:
df_train = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')

In [94]:
df_train.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [95]:
df_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [96]:
df_train.columns.size

19

# Q2. Computing duration

In [97]:
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train.duration

0         0 days 00:08:26
1         0 days 00:06:19
2         0 days 00:12:45
3         0 days 00:09:37
4         0 days 00:10:50
                ...      
3066761   0 days 00:13:59
3066762   0 days 00:19:27
3066763   0 days 00:24:31
3066764   0 days 00:13:00
3066765   0 days 00:14:24
Name: duration, Length: 3066766, dtype: timedelta64[us]

In [98]:
df_train.duration = df_train.duration.apply(lambda x: x.total_seconds()/60)

In [99]:
df_train.duration

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [100]:
df_train.duration.std()

42.59435124195458

# Q3. Dropping outliers

In [101]:
total = df_train.shape[0]

In [102]:
total

3066766

In [103]:
records_in_1_60_min_range = df_train[ (df_train.duration >= 1) & (df_train.duration <= 60) ].shape[0]

In [104]:
records_in_1_60_min_range

3009173

In [105]:
percentage_in_range = round(records_in_1_60_min_range/total, 2)

In [106]:
percentage_in_range

0.98

In [108]:
df_train = df_train[ (df_train.duration >= 1) & (df_train.duration <= 60) ]

In [109]:
df_train.shape

(3009173, 20)

# Q4. One-hot encoding

In [123]:
#PULocationID	DOLocationID
categories = ['PULocationID', 'DOLocationID']
df_train[categories] = df_train[categories].astype(str)
train_data = df_train[categories].to_dict(orient='records')
df_train[categories].shape

(3009173, 2)

In [124]:
dv = DictVectorizer()
x_train = dv.fit_transform(train_data)
x_train.shape

(3009173, 515)

# Q5. Training a model

In [125]:
y_train = df_train['duration'].values
lr = LinearRegression()
lr.fit(x_train,y_train)
y_train_pred = lr.predict(x_train)

In [126]:
root_mean_squared_error(y_train,y_train_pred)

7.649261929201487

# Q6. Evaluating the model

In [130]:
# Now let's apply this model to the validation dataset (February 2023).

# What's the RMSE on validation?

# 3.81
# 7.81
# 11.81
# 16.81

df_val = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda x: x.total_seconds()/60)
df_val = df_val[ (df_val.duration >= 1) & (df_val.duration <= 60) ]

#PULocationID	DOLocationID
categories = ['PULocationID', 'DOLocationID']
df_val[categories] = df_val[categories].astype(str)
val_data = df_val[categories].to_dict(orient='records')

In [134]:
x_val = dv.fit(val_data)

In [135]:
y_val = df_val['duration'].values
y_val_pred = lr.predict(x_val)

AttributeError: 'DictVectorizer' object has no attribute 'ndim'

In [ ]:
root_mean_squared_error(y_val,y_val_pred)